In [1]:
using DelimitedFiles
using Flux
using Flux: onehotbatch, crossentropy, onecold
using Statistics: mean

In [2]:
0))# https://www.kaggle.com/nltkdata/qc-corpus
function load_dataset(mode="train")
    dataset = readdlm("$mode.txt", ':', String, '\n')
    labels = dataset[:,1]
    questions = [split(lowercase(ques))[2:end] for ques in dataset[:, 2]]
    
    return (questions, labels)
end

load_dataset (generic function with 2 methods)

In [19]:
Train_questions, Train_labels = load_dataset()
frequencyDict = Dict{String, Number}()
words = String[]

for ques in Train_questions
    for token in ques
        if token ∉ words
            push!(words,token)
        end
        if token ∉ keys(frequencyDict)
            frequencyDict[token] = 0
        end
            frequencyDict[token] += 1
    end
end

function countWords(str)
    frequencyDict[str]
end

words = sort(words, by=countWords, rev=true)[1:200]
# StopWords?
push!(words, "<OTHER>") # To make computation easier

201-element Array{String,1}:
 "?"         
 "the"       
 "what"      
 "is"        
 "of"        
 "in"        
 "a"         
 "how"       
 "'s"        
 "was"       
 "who"       
 "to"        
 ","         
 ⋮           
 "die"       
 "animal"    
 "sea"       
 "school"    
 "york"      
 "over"      
 "history"   
 "part"      
 "great"     
 "money"     
 "university"
 "<OTHER>"   

In [29]:
function inputFeatures(tokens)
    features = zeros(length(words))
    for token in tokens
        index = findfirst(isequal(token),words)
        if index == nothing
            index = length(words)
        end
        features[index] = 1
    end
    return features
end
Xs = hcat([inputFeatures(ques) for ques in Train_questions]...)
Ys = (onehotbatch(Train_labels, unique(Train_labels)))

6×5452 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
  true  false   true  false  false  …  false  false  false  false  false
 false   true  false   true  false      true   true  false  false   true
 false  false  false  false   true     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false   true   true  false
 false  false  false  false  false  …  false  false  false  false  false

In [46]:
m = Chain(
    Dense(length(words), 32, relu),
    Dense(32, length(unique(Train_labels))),
    softmax)

Chain(Dense(201, 32, NNlib.relu), Dense(32, 6), NNlib.softmax)

In [47]:
loss(x, y) = crossentropy(m(x), y)
accuracy(x,y) = mean(onecold(m(x)) .== onecold(y))
dataset = Iterators.repeated((Xs, Ys), 500)
opt = ADAM(0.01)

evalcb = () -> @show(accuracy(Xs, Ys))

#27 (generic function with 1 method)

In [48]:
Flux.train!(loss, params(m), dataset, opt, cb = Flux.throttle(evalcb, 1))

accuracy(Xs, Ys) = 0.2619222303741746
accuracy(Xs, Ys) = 0.8138297872340425
accuracy(Xs, Ys) = 0.8679383712399119
accuracy(Xs, Ys) = 0.8985693323550991
accuracy(Xs, Ys) = 0.9169112252384446
accuracy(Xs, Ys) = 0.925531914893617
accuracy(Xs, Ys) = 0.9306676449009538
